**Cat Facter Agent**


In [ ]:
import torch
import json
import requests
import time
from transformers import AutoTokenizer, AutoModelForCausalLM
from datetime import datetime

class SwarmBrain(torch.nn.Module):
    def __init__(self, model_name="HuggingFaceTB/SmolLM2-135M"):
        super(SwarmBrain, self).__init__()
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name, torch_dtype=torch.bfloat16
        ).to("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def make_decision(self, context):
        prompt = f"""As an AI swarm coordinator, analyze this context and make a decision:

        Context:
        {context}

        Decision needed: What should the swarm do next?

        Response format:
        Action: [specific action to take]
        Reason: [explanation]
        Parameters: [any specific parameters needed]"""

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(
            inputs["input_ids"],
            max_length=200,
            temperature=0.7,
            return_dict_in_generate=True,
            output_scores=True
        )

        decision = self.tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
        confidence = torch.mean(torch.stack([torch.max(score) for score in outputs.scores])).item()

        return decision, confidence

class SwarmAgent:
    def __init__(self, api_url):
        self.api_url = api_url

    def get_cat_fact(self):
        """Make an API call to retrieve a random cat fact"""
        try:
            response = requests.get(self.api_url)
            if response.status_code == 200:
                return response.json().get("data")[0]  # Extract the first cat fact
            return "No cat fact retrieved."
        except Exception as e:
            return f"Error: {e}"

class CatFactCollector:
    def __init__(self, api_url, num_facts=3):
        self.agents = [SwarmAgent(api_url) for _ in range(num_facts)]

    def get_cat_facts(self):
        """Retrieve a list of cat facts"""
        facts = [agent.get_cat_fact() for agent in self.agents]
        return facts

    def save_facts_to_notepad(self, facts):
        """Save the cat facts to a Notepad file"""
        filename = f"CatFacts_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
        with open(filename, "w") as file:
            file.write("Cat Facts:\n\n")
            for i, fact in enumerate(facts, 1):
                file.write(f"{i}. {fact}\n")
        print(f"Cat facts saved to {filename}")

if __name__ == "__main__":
    # URL for the cat facts API
    api_url = "https://meowfacts.herokuapp.com/"

    # Initialize the cat fact collector and get facts
    collector = CatFactCollector(api_url)
    facts = collector.get_cat_facts()

    # Save the facts to a Notepad file
    collector.save_facts_to_notepad(facts)

**Bored Agent**

In [ ]:
import torch
import json
import requests
from transformers import AutoTokenizer, AutoModelForCausalLM
from datetime import datetime
import time

class SwarmBrain(torch.nn.Module):
    def __init__(self, model_name="HuggingFaceTB/SmolLM2-135M"):
        super(SwarmBrain, self).__init__()
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name, torch_dtype=torch.bfloat16
        ).to("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def make_decision(self, context):
        prompt = f"""As an AI swarm coordinator, analyze this context and make a decision:

        Context:
        {context}

        Decision needed: What should the swarm do next?

        Response format:
        Action: [specific action to take]
        Reason: [explanation]
        Parameters: [any specific parameters needed]"""

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(
            inputs["input_ids"],
            max_length=200,
            temperature=0.7,
            return_dict_in_generate=True,
            output_scores=True
        )

        decision = self.tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
        confidence = torch.mean(torch.stack([torch.max(score) for score in outputs.scores])).item()

        return decision, confidence

    def generate_essay(self, activities):
        """Generate a short essay about the activities"""
        context = "Here are some interesting activities to try:\n\n" + "\n".join(activities)
        prompt = f"""Write a short essay about the following activities and their potential benefits:

        {context}

        Essay:"""

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(
            inputs["input_ids"],
            max_length=500,
            temperature=0.7,
            return_dict_in_generate=True,
            output_scores=True
        )

        essay = self.tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
        return essay

class SwarmAgent:
    def __init__(self, api_url, retries=3):
        self.api_url = api_url
        self.retries = retries

    def get_activity(self):
        """Make an API call to retrieve a random activity with retries"""
        for attempt in range(self.retries):
            try:
                response = requests.get(self.api_url, timeout=5)
                if response.status_code == 200:
                    return response.json().get("activity", "No activity available.")
                else:
                    print(f"Attempt {attempt+1} failed with status code {response.status_code}. Retrying...")
            except requests.RequestException as e:
                print(f"Attempt {attempt+1} encountered an error: {e}. Retrying...")
            time.sleep(1)  # wait before retrying
        return "No activity retrieved after multiple attempts."

class ActivityCollector:
    def __init__(self, api_url, num_activities=3):
        self.agents = [SwarmAgent(api_url) for _ in range(num_activities)]
        self.brain = SwarmBrain()

    def get_activities(self):
        """Retrieve a list of random activities"""
        activities = [agent.get_activity() for agent in self.agents]
        return activities

    def save_activities_to_notepad(self, activities):
        """Save the activities and a generated essay to a Notepad file"""
        essay = self.brain.generate_essay(activities)
        filename = f"Activities_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"

        with open(filename, "w") as file:
            file.write("Suggested Activities:\n\n")
            for i, activity in enumerate(activities, 1):
                file.write(f"{i}. {activity}\n")
            file.write("\n\nGenerated Essay:\n\n")
            file.write(essay)

        print(f"Activities and essay saved to {filename}")

if __name__ == "__main__":
    # URL for the alternative Bored API
    api_url = "https://bored-api.appbrewery.com/random"

    # Initialize the activity collector and get activities
    collector = ActivityCollector(api_url)
    activities = collector.get_activities()

    # Save the activities and the essay to a Notepad file
    collector.save_activities_to_notepad(activities)

**Multi Agent Story Generator**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datetime import datetime
import random

class SwarmBrain(torch.nn.Module):
    def __init__(self, model_name="HuggingFaceTB/SmolLM2-135M"):
        super(SwarmBrain, self).__init__()
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name
        ).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        # Ensure pad token is set
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def integrate_story(self, contributions, title):
        """Integrate agents' contributions into a coherent story."""
        contributions_text = "\n\n".join(contributions)
        prompt = f"""Title: {title}

Agents' Contributions:
{contributions_text}

As the SwarmBrain, integrate the above contributions into a coherent and engaging story. Ensure the story flows well, characters are consistent, and the plot is compelling.

Final Story:"""

        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True).to(self.device)
        outputs = self.model.generate(
            inputs["input_ids"],
            max_new_tokens=500,
            temperature=0.7,
            num_beams=5,
            early_stopping=True,
            no_repeat_ngram_size=2,
            pad_token_id=self.tokenizer.eos_token_id
        )

        story = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract the final story after "Final Story:"
        story = story.split("Final Story:")[-1].strip()
        return story

class SwarmAgent:
    def __init__(self, agent_id, model_name="HuggingFaceTB/SmolLM2-135M"):
        self.agent_id = agent_id
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name
        ).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        # Ensure pad token is set
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def write_contribution(self, context):
        """Agent writes a paragraph based on the provided context."""
        prompt = f"""As Agent {self.agent_id}, continue the story based on the context below. Write a cohesive and engaging paragraph that builds upon the existing narrative.

Context:
{context}

Agent {self.agent_id}'s Contribution:"""

        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True).to(self.device)
        outputs = self.model.generate(
            inputs["input_ids"],
            max_new_tokens=150,
            temperature=0.7,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            early_stopping=True,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id
        )

        contribution = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract the agent's contribution after "Agent X's Contribution:"
        contribution = contribution.split(f"Agent {self.agent_id}'s Contribution:")[-1].strip()
        return contribution

class StoryCoordinator:
    def __init__(self, num_agents=3):
        self.agents = [SwarmAgent(agent_id=i+1) for i in range(num_agents)]
        self.brain = SwarmBrain()
        self.title = self.generate_story_title()

    def generate_story_title(self):
        """Generate a random story title."""
        titles = [
            "The Lost Artifact",
            "Echoes of the Past",
            "Journey to the Unknown",
            "Shadows in the Forest",
            "The Time Traveler's Dilemma"
        ]
        return random.choice(titles)

    def coordinate_story(self):
        """Coordinate agents to create a story."""
        context = f"Title: {self.title}\n\nOnce upon a time,"
        contributions = []

        for agent in self.agents:
            contribution = agent.write_contribution(context)
            contributions.append(contribution)
            # Update context for the next agent
            context += "\n\n" + contribution

        final_story = self.brain.integrate_story(contributions, self.title)
        return final_story

    def save_story(self, story):
        """Save the final story to a file."""
        filename = f"Story_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
        with open(filename, "w", encoding='utf-8') as file:
            file.write(f"Title: {self.title}\n\n")
            file.write(story)
        print(f"Story saved to {filename}")

if __name__ == "__main__":
    # Initialize the story coordinator
    coordinator = StoryCoordinator(num_agents=3)

    # Coordinate the agents to create a story
    final_story = coordinator.coordinate_story()

    # Save the final story to a file
    coordinator.save_story(final_story)